In [1]:
!git clone https://github.com/VuThanhLam124/Profiling_gender_dialect.git

Cloning into 'Profiling_gender_dialect'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 53 (delta 23), reused 49 (delta 19), pack-reused 0 (from 0)
Receiving objects: 100% (53/53), 47.65 KiB | 3.67 MiB/s, done.
Resolving deltas: 100% (23/23), done.


In [2]:
cd Profiling_gender_dialect

/kaggle/working/Profiling_gender_dialect


In [3]:
ls

app.py    eval.py      infer.py    prepare_data.py  requirements.txt
configs/  finetune.py  notebooks/  README.md        src/


In [4]:
cd configs

/kaggle/working/Profiling_gender_dialect/configs


In [5]:
ls

eval.yaml          finetune.yaml          infer.yaml
eval.yaml.example  finetune.yaml.example  infer.yaml.example


In [6]:
cd finetune.yaml

[Errno 20] Not a directory: 'finetune.yaml'
/kaggle/working/Profiling_gender_dialect/configs


/bin/bash: line 1: nano: command not found
